## 1. Cargar datos procesados

In [1]:
import pickle
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import matplotlib.colors as mcolors

# Cargar datos procesados
with open('datos_procesados.pkl', 'rb') as f:
    datos = pickle.load(f)

habilidades = datos['habilidades']
grupos_bge_ngram = datos['grupos_bge_ngram']
tfidf_epn_69d = datos['tfidf_epn_69d']

print("✓ Datos cargados correctamente")
print(f"\nCarreras académicas disponibles: {len(tfidf_epn_69d.columns)}")

✓ Datos cargados correctamente

Carreras académicas disponibles: 24


## 2. Definir carreras a analizar y cargar ofertas laborales
Especifica qué carreras quieres incluir en la visualización interactiva

In [2]:
# Configuración de carreras a analizar
# Formato: (nombre_académico, nombre_visualización, ruta_csv)
CARRERAS_CONFIG = [
    ('Ingenieria En Ciencias De La Computacion', 'Computación', 'todas_las_plataformas/Computación/Computación_Merged.csv'),
    ('Ciencias De Datos E Inteligencia Artificial', 'Ciencias De Datos E IA', 'todas_las_plataformas/Ciencia_de_Datos/Ciencia_de_Datos_Merged.csv'),
    ('Ingenieria En Software', 'Software', 'todas_las_plataformas/Software/Software_Merged.csv'),
    ('Ingenieria En Sistemas De Informacion', 'Sistemas de Información', 'todas_las_plataformas/Sistemas_de_Información/Sistemas_de_Información_Merged.csv'),
    ('Licenciatura Administracion De Empresas', 'Administración de Empresas', 'todas_las_plataformas/Administración_de_Empresas/Administración_de_Empresas_Merged.csv'),
    ('Ingenieria Agroindustria', 'Agroindustria', 'todas_las_plataformas/Agroindustria/Agroindustria_Merged.csv'),
    ('Matematica', 'Matemática', 'todas_las_plataformas/Matemática/Matemática_Merged.csv'),
    ('Matematica Aplicada', 'Matemática Aplicada', 'todas_las_plataformas/Matemática_Aplicada/Matemática_Aplicada_Merged.csv'),
    ('Fisica', 'Física', 'todas_las_plataformas/Física/Física_Merged.csv'),
    ('Ingenieria En Geologia', 'Geología', 'todas_las_plataformas/Geología/Geología_Merged.csv'),
    ('Ingenieria De La Produccion', 'Ingeniería De La Producción', 'todas_las_plataformas/Ingeniería_de_la_Producción/Ingeniería_de_la_Producción_Merged.csv'),
    ('Ingenieria En Materiales', 'Materiales', 'todas_las_plataformas/Materiales/Materiales_Merged.csv'),
    ('Ingenieria En Mecanica', 'Mecánica', 'todas_las_plataformas/Mecánica/Mecánica_Merged.csv'),
    ('Ingenieria En Mecatronica', 'Mecatrónica', 'todas_las_plataformas/Mecatrónica/Mecatrónica_Merged.csv'),
    ('Ingenieria En Petroleos', 'Petróleos', 'todas_las_plataformas/Petróleos/Petróleos_Merged.csv'),
    ('Ingenieria Quimica', 'Ingeniería Química', 'todas_las_plataformas/Ingeniería_Química/Ingeniería_Química_Merged.csv'),
    ('Ingenieria En Telecomunicaciones', 'Telecomunicaciones', 'todas_las_plataformas/Telecomunicaciones/Telecomunicaciones_Merged.csv'),
    ('Ingenieria Civil', 'Ingeniería Civil', 'todas_las_plataformas/Ingeniería_Civil/Ingeniería_Civil_Merged.csv'),
    ('Economia', 'Economía', 'todas_las_plataformas/Economía/Economía_Merged.csv'),
    ('Ingenieria En Electricidad', 'Electricidad', 'todas_las_plataformas/Electricidad/Electricidad_Merged.csv'),
    ('Ingenieria En Electronica Y Automatizacion', 'Electrónica Y Automatización', 'todas_las_plataformas/Electrónica_y_Automatización/Electrónica_y_Automatización_Merged.csv'),
    ('Ingenieria Ambiental', 'Ingeniería Ambiental', 'todas_las_plataformas/Ingeniería_Ambiental/Ingeniería_Ambiental_Merged.csv'),
]

print(f"Se analizarán {len(CARRERAS_CONFIG)} carreras")

Se analizarán 22 carreras


## 3. Procesar ofertas laborales para todas las carreras

In [3]:
def procesar_ofertas_carrera(ruta_csv):
    """Vectoriza las ofertas laborales de una carrera."""
    try:
        df = pd.read_csv(ruta_csv, dtype=str)
        textos = df[['skills','description']].fillna('').agg(' '.join, axis=1).str.lower().tolist()
        
        # Vectorizar
        vectorizer = CountVectorizer(vocabulary=habilidades, analyzer='word', ngram_range=(1, 5), lowercase=True)
        X = vectorizer.transform(textos)
        matriz_td = pd.DataFrame(X.T.toarray(), index=vectorizer.get_feature_names_out())
        
        # Agrupar por dimensiones
        matriz_69d = pd.DataFrame(0, index=grupos_bge_ngram.keys(), columns=range(len(textos)))
        for label, terms in grupos_bge_ngram.items():
            matriz_69d.loc[label] = matriz_td.loc[terms].sum(axis=0)
        
        matriz_69d.index = [', '.join(grupos_bge_ngram[label][:3]) + (' ...' if len(grupos_bge_ngram[label])>3 else '') 
                           for label in grupos_bge_ngram.keys()]
        
        # TF-IDF
        tfidf_transformer = TfidfTransformer(norm='l2').fit(matriz_69d.values)
        tfidf_69d = pd.DataFrame(
            tfidf_transformer.transform(matriz_69d.values).toarray(), 
            index=matriz_69d.index, 
            columns=matriz_69d.columns
        ).T
        
        return tfidf_69d
    except Exception as e:
        print(f"❌ Error procesando {ruta_csv}: {e}")
        return None

# Procesar todas las carreras
vectores_academicos = []
vectores_laborales = []
nombres_carreras = []

print("Procesando ofertas laborales...\n")
for nombre_acad, nombre_vis, ruta_csv in CARRERAS_CONFIG:
    if nombre_acad not in tfidf_epn_69d.columns:
        print(f"⚠ '{nombre_acad}' no encontrado en datos académicos, se omite")
        continue
    
    tfidf_ofertas = procesar_ofertas_carrera(ruta_csv)
    if tfidf_ofertas is None:
        continue
    
    vector_acad = tfidf_epn_69d.T.loc[[nombre_acad]]
    
    vectores_academicos.append((nombre_vis, vector_acad))
    vectores_laborales.append((nombre_vis, tfidf_ofertas))
    nombres_carreras.append(nombre_vis)
    
    print(f"✓ {nombre_vis}: {len(tfidf_ofertas)} ofertas")

print(f"\n✓ Procesadas {len(nombres_carreras)} carreras correctamente")

Procesando ofertas laborales...

✓ Computación: 4101 ofertas
✓ Computación: 4101 ofertas
✓ Ciencias De Datos E IA: 4836 ofertas
✓ Ciencias De Datos E IA: 4836 ofertas
✓ Software: 5873 ofertas
✓ Software: 5873 ofertas
✓ Sistemas de Información: 6157 ofertas
✓ Sistemas de Información: 6157 ofertas
✓ Administración de Empresas: 5220 ofertas
✓ Administración de Empresas: 5220 ofertas
✓ Agroindustria: 1688 ofertas
✓ Agroindustria: 1688 ofertas
✓ Matemática: 399 ofertas
✓ Matemática: 399 ofertas
✓ Matemática Aplicada: 383 ofertas
✓ Matemática Aplicada: 383 ofertas
✓ Física: 1614 ofertas
✓ Física: 1614 ofertas
✓ Geología: 1074 ofertas
✓ Geología: 1074 ofertas
✓ Ingeniería De La Producción: 4503 ofertas
✓ Ingeniería De La Producción: 4503 ofertas
✓ Materiales: 2127 ofertas
✓ Materiales: 2127 ofertas
✓ Mecánica: 1633 ofertas
✓ Mecánica: 1633 ofertas
✓ Mecatrónica: 1338 ofertas
✓ Mecatrónica: 1338 ofertas
✓ Petróleos: 1551 ofertas
✓ Petróleos: 1551 ofertas
✓ Ingeniería Química: 1148 ofertas
✓ In

## 4. Calcular PCA global (una sola vez para todas las carreras)
Esto asegura que todas las carreras estén en el mismo espacio 2D

In [4]:
# Preparar datos para PCA global
colores_base = list(mcolors.TABLEAU_COLORS.values()) + list(mcolors.CSS4_COLORS.values())

df_academicos = []
df_laborales = []
labels_academicos = []
labels_laborales = []
colores_academicos = []
colores_laborales = []

# Procesar vectores académicos
for i, (carrera, vector_acad) in enumerate(vectores_academicos):
    if hasattr(vector_acad, 'values'):
        df_academicos.append(vector_acad.values[0])
    else:
        df_academicos.append(vector_acad)
    labels_academicos.append(carrera)
    color = colores_base[i % len(colores_base)]
    colores_academicos.append(color)

# Procesar vectores laborales
for i, (carrera, tfidf_lab_69d) in enumerate(vectores_laborales):
    if hasattr(tfidf_lab_69d, 'iloc'):
        for j in range(tfidf_lab_69d.shape[0]):
            df_laborales.append(tfidf_lab_69d.iloc[j].values)
            labels_laborales.append(carrera)
            color = colores_base[i % len(colores_base)]
            colores_laborales.append(mcolors.to_rgba(color, alpha=0.3))
    else:
        for j in range(tfidf_lab_69d.shape[0]):
            df_laborales.append(tfidf_lab_69d[j])
            labels_laborales.append(carrera)
            color = colores_base[i % len(colores_base)]
            colores_laborales.append(mcolors.to_rgba(color, alpha=0.3))

# Unir todo y aplicar PCA
df_total = np.vstack(df_academicos + df_laborales)
labels_total = labels_academicos + labels_laborales
colores_total = colores_academicos + colores_laborales

print("Aplicando PCA...")
pca_global = PCA(n_components=2)
pca_result_global = pca_global.fit_transform(df_total)

print(f"✓ PCA completado")
print(f"  - Varianza explicada PC1: {pca_global.explained_variance_ratio_[0]*100:.1f}%")
print(f"  - Varianza explicada PC2: {pca_global.explained_variance_ratio_[1]*100:.1f}%")
print(f"  - Total puntos: {len(pca_result_global)}")

Aplicando PCA...
✓ PCA completado
  - Varianza explicada PC1: 4.4%
  - Varianza explicada PC2: 3.2%
  - Total puntos: 64682


## 5. Crear visualización interactiva optimizada con selector de carrera
**OPTIMIZACIÓN CLAVE**: En lugar de mostrar todas las carreras simultáneamente (miles de trazas),
usamos el parámetro `visible` de Plotly para mostrar solo una carrera a la vez.

In [5]:
# Crear figura interactiva con Plotly
fig = go.Figure()

offset_academico = len(df_academicos)

# Agregar trazas para cada carrera (académico + laborales)
# CLAVE: Todas empiezan como invisible=False para la primera carrera, resto True
for i, carrera in enumerate(nombres_carreras):
    # Punto académico (estrella)
    idx_acad = i
    fig.add_trace(go.Scatter(
        x=[pca_result_global[idx_acad, 0]],
        y=[pca_result_global[idx_acad, 1]],
        mode='markers',
        marker=dict(
            symbol='star',
            size=16,
            color=colores_academicos[i],
            line=dict(width=2, color='black')
        ),
        name=f'{carrera} (académico)',
        legendgroup=carrera,
        hovertemplate=f'<b>{carrera}</b><br>Académico<br>PC1: %{{x:.3f}}<br>PC2: %{{y:.3f}}<extra></extra>',
        visible=(i == 0)  # Solo la primera carrera visible al inicio
    ))
    
    # Puntos laborales (círculos)
    indices_laborales = [j for j, label in enumerate(labels_laborales) if label == carrera]
    indices_laborales_global = [offset_academico + idx for idx in indices_laborales]
    
    fig.add_trace(go.Scatter(
        x=pca_result_global[indices_laborales_global, 0],
        y=pca_result_global[indices_laborales_global, 1],
        mode='markers',
        marker=dict(
            symbol='circle',
            size=6,
            color=colores_academicos[i],
            opacity=0.4
        ),
        name=f'{carrera} (laboral)',
        legendgroup=carrera,
        hovertemplate=f'<b>{carrera}</b><br>Oferta laboral<br>PC1: %{{x:.3f}}<br>PC2: %{{y:.3f}}<extra></extra>',
        showlegend=False,
        visible=(i == 0)  # Solo la primera carrera visible al inicio
    ))

print(f"✓ Creadas {len(fig.data)} trazas ({len(nombres_carreras) * 2})")

✓ Creadas 44 trazas (44)


## 6. Agregar botones de selección de carrera

In [6]:
# Crear botones para selector dropdown
buttons = []

for i, carrera in enumerate(nombres_carreras):
    # Crear máscara de visibilidad: solo las 2 trazas de esta carrera son visibles
    visibility = [False] * len(fig.data)
    visibility[i * 2] = True      # Punto académico
    visibility[i * 2 + 1] = True  # Puntos laborales
    
    buttons.append(dict(
        label=carrera,
        method='update',
        args=[{'visible': visibility},
              {'title': f'PCA: {carrera} - Carrera vs Mercado laboral'}]
    ))

# Botón para mostrar todas las carreras
buttons.append(dict(
    label='Todas las carreras',
    method='update',
    args=[{'visible': [True] * len(fig.data)},
          {'title': 'PCA global: Todas las Carreras vs Mercado laboral'}]
))

print(f"✓ Creados {len(buttons)} botones de selección")

✓ Creados 23 botones de selección


## 7. Configurar layout y exportar

In [7]:
# Configurar layout con dropdown
fig.update_layout(
    title=f'PCA: {nombres_carreras[0]} - Carrera vs Mercado laboral',
    xaxis_title=f'PCA 1 ({pca_global.explained_variance_ratio_[0]*100:.1f}% varianza)',
    yaxis_title=f'PCA 2 ({pca_global.explained_variance_ratio_[1]*100:.1f}% varianza)',
    width=1000,
    height=700,
    template='plotly_white',
    hovermode='closest',
    updatemenus=[
        dict(
            buttons=buttons,
            direction='down',
            pad={'r': 10, 't': 10},
            showactive=True,
            x=0.01,
            xanchor='left',
            y=1.15,
            yanchor='top',
            bgcolor='rgba(255, 255, 255, 0.9)',
            bordercolor='#CCCCCC',
            borderwidth=1
        )
    ],
    annotations=[
        dict(
            text='Seleccionar carrera:',
            showarrow=False,
            x=0.01,
            y=1.18,
            xref='paper',
            yref='paper',
            align='left',
            xanchor='left',
            yanchor='top',
            font=dict(size=12, color='#333333')
        )
    ]
)

# Mostrar en notebook
fig.show()

# Exportar a HTML
nombre_archivo = 'PCA_global.html'
pio.write_html(fig, nombre_archivo, auto_open=False)

print(f"\n{'='*80}")
print(f"✓ Visualización completada exitosamente")
print(f"{'='*80}")
print(f"\nEstadísticas:")
print(f"   - Carreras analizadas: {len(nombres_carreras)}")
print(f"   - Total de puntos: {len(pca_result_global):,}")
print(f"   - Trazas Plotly: {len(fig.data)}")
print(f"\nArchivo generado: {nombre_archivo}")


✓ Visualización completada exitosamente

Estadísticas:
   - Carreras analizadas: 22
   - Total de puntos: 64,682
   - Trazas Plotly: 44

Archivo generado: PCA_global.html
